In [1]:
from scprint import scPrint
from scdataloader import Preprocessor, utils
from scprint.tasks import GNInfer, Embedder, Denoiser, withknn

from bengrn import BenGRN, get_sroy_gt, compute_genie3
from bengrn.base import train_classifier

from grnndata import utils as grnutils
from grnndata import read_h5ad

from anndata.utils import make_index_unique
from anndata import concat
import scanpy as sc

from matplotlib import pyplot as plt
import numpy as np
import lamindb as ln

%load_ext autoreload
%autoreload 2 

import torch
torch.set_float32_matmul_precision('medium')

💡 connected lamindb: annamaguza/scprint


In [2]:
adata = sc.read_h5ad('/mnt/LaCIE/annaM/gut_project/Processed_data/Gut_data/Fetal_stem_cells/Fetal_cells_filtered_raw.h5ad')

* Preprocessing using scDataloader

In [10]:
cx_dataset = ln.Collection.using(instance="laminlabs/cellxgene").filter(name="cellxgene-census", version='2023-12-15').one()

In [11]:
prostate_adata = cx_dataset.artifacts.filter(key='cell-census/2023-12-15/h5ads/574e9f9e-f8b4-41ef-bf19-89a9964fd9c7.h5ad').one().load()

... synchronizing 574e9f9e-f8b4-41ef-bf19-89a9964fd9c7.h5ad: 100.0%


In [12]:
prostate_adata.obs

,Sample,Lineage,Population,resolution_0.1,resolution_0.2,resolution_0.3,resolution_0.4,resolution_0.5,resolution_0.75,resolution_1,...,donor_id,suspension_type,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage
AAACCTGAGAATCTCC_1,D17PrPzF_Via,Epithelia,BE,0,0,0,0,0,0,2,...,D17,cell,basal cell of prostate epithelium,10x 3' v2,normal,Homo sapiens,male,peripheral zone of prostate,European,31-year-old human stage
AAACCTGAGAGACGAA_1,D17PrPzF_Via,Epithelia,BE,0,0,2,1,1,1,0,...,D17,cell,basal cell of prostate epithelium,10x 3' v2,normal,Homo sapiens,male,peripheral zone of prostate,European,31-year-old human stage
AAACCTGAGCGGATCA_1,D17PrPzF_Via,Epithelia,Club,2,2,3,8,10,10,11,...,D17,cell,secretory cell,10x 3' v2,normal,Homo sapiens,male,peripheral zone of prostate,European,31-year-old human stage
AAACCTGAGGTAGCTG_1,D17PrPzF_Via,Epithelia,BE,0,0,0,0,0,0,5,...,D17,cell,basal cell of prostate epithelium,10x 3' v2,normal,Homo sapiens,male,peripheral zone of prostate,European,31-year-old human stage
AAACCTGAGTTGTCGT_1,D17PrPzF_Via,Epithelia,Club,0,0,0,4,12,3,13,...,D17,cell,secretory cell,10x 3' v2,normal,Homo sapiens,male,peripheral zone of prostate,European,31-year-old human stage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGAGACTTCCA_19,BPH556PrGA2_Fcol,Endothelia,Endothelia,1,1,1,2,9,11,10,...,BPH556,cell,prostate gland microvascular endothelial cell,10x 3' v3,benign prostatic hyperplasia,Homo sapiens,male,transition zone of prostate,European,71-year-old human stage
TTTGTTGAGGTACTGG_19,BPH556PrGA2_Fcol,Endothelia,Endothelia,1,1,1,2,2,2,1,...,BPH556,cell,prostate gland microvascular endothelial cell,10x 3' v3,benign prostatic hyperplasia,Homo sapiens,male,transition zone of prostate,European,71-year-old human stage
TTTGTTGCAGCAGACA_19,BPH556PrGA2_Fcol,Myeloid,IGSF21+ Dendritic,3,4,5,11,14,15,18,...,BPH556,cell,dendritic cell,10x 3' v3,benign prostatic hyperplasia,Homo sapiens,male,transition zone of prostate,European,71-year-old human stage
TTTGTTGTCCCATTTA_19,BPH556PrGA2_Fcol,Endothelia,Endothelia,1,1,1,2,2,2,1,...,BPH556,cell,prostate gland microvascular endothelial cell,10x 3' v3,benign prostatic hyperplasia,Homo sapiens,male,transition zone of prostate,European,71-year-old human stage


In [ ]:
adata.obs['organism_ontology_term_id']='NCBITaxon:9606'

In [8]:
preprocessor = Preprocessor(do_postp=False)
adata = preprocessor(adata)

Dropping layers:  KeysView(Layers with keys: )
checking raw counts
removed 0 non primary cells, 231646 renamining


AttributeError: 'NoneType' object has no attribute 'id'